<a href="https://colab.research.google.com/github/AbhimanyuAryan/Deep_Learning_Computer_Vision/blob/master/training_model_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing library to download dataset
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   321  100   321    0     0   1621      0 --:--:-- --:--:-- --:--:--  1621
Updating fastai...
Done.


In [0]:
import torch
import torch.nn.functional as F
from fastai import datasets
import gzip, pickle, matplotlib as mpl
from torch import tensor
from torch import nn

In [0]:
mpl.rcParams['image.cmap'] = 'gray'

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

In [0]:
n, m = x_train.shape
c = y_train.max() + 1
nh = 50

In [0]:
class Model(nn.Module):
  def __init__(self, n_in, nh, n_out):
    super().__init__()
    self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
  
  def __call__(self, x):
    for l in self.layers: x = l(x)
    return x

In [32]:
print(n, m, c, nh)

50000 784 tensor(10) 50


In [0]:
model = Model(m, nh, 10)

In [0]:
pred = model(x_train)

In [37]:
pred

tensor([[ 0.1340,  0.1256,  0.1134,  ..., -0.0683, -0.0565, -0.0414],
        [ 0.1201,  0.1072,  0.1145,  ..., -0.1127, -0.0268, -0.0263],
        [-0.0385,  0.0783,  0.0556,  ...,  0.0033, -0.0843, -0.1203],
        ...,
        [ 0.0287,  0.1308,  0.0793,  ..., -0.0120, -0.0256, -0.1167],
        [-0.0121,  0.1052, -0.0094,  ..., -0.0625, -0.0358, -0.0877],
        [-0.0707,  0.1246,  0.0465,  ..., -0.0248,  0.0124, -0.1041]],
       grad_fn=<AddmmBackward>)

### Cross Entropy

In [0]:

def log_softmax(x): return (x.exp()/(x.exp().sum(-1, keepdim=True))).log()

In [0]:
sm_pred = log_softmax(pred)

In [40]:
y_train[:3]

tensor([5, 0, 4])

In [41]:
sm_pred[[0,1,2], [5,0,4]]

tensor([-2.3310, -2.2229, -2.3391], grad_fn=<IndexBackward>)

In [42]:
y_train.shape[0]

50000

In [0]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [0]:
loss = nll(sm_pred, y_train)

### Basic training LOOP

In [0]:
loss_func = F.cross_entropy

In [0]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [56]:
bs = 64

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)
preds[0], preds.shape   # preds will have possible probibilities

(tensor([ 0.1340,  0.1256,  0.1134,  0.0995, -0.0159,  0.0204,  0.1439, -0.0683,
         -0.0565, -0.0414], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [57]:
yb = y_train[0:bs]
loss_func(preds, yb)

tensor(2.3022, grad_fn=<NllLossBackward>)

In [58]:
accuracy(preds, yb)

tensor(0.1406)

In [0]:
lr = 0.5
epochs = 1

In [0]:
for epoch in range(epochs):
  for i in range((n-1)//bs + 1):
    start_i = i*bs
    end_i = start_i + bs
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]
    loss = loss_func(model(xb), yb)

    loss.backward()
    with torch.no_grad():
      for l in model.layers:
        if hasattr(l, 'weight'):
          l.weight -= l.weight.grad * lr
          l.bias -= l.bias.grad * lr
          l.weight.grad.zero_()
          l.bias.grad.zero_()

In [61]:
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.1050, grad_fn=<NllLossBackward>), tensor(0.9375))

### using params and optim

In [0]:
class DummyModule():
  def __init__(self, n_in, nh, n_out):
    self._modules = {}
    self.l1 = nn.Linear(n_in, nh)
    self.l2 = nn.Linear(nh, n_out)

  def __setattr__(self,k,v):
    if not k.startswith("_"): self._modules[k] = v
  
  def __repr__(self): return f'{self._modules}'

  def parameters(self):
    for l in self._modules.values():
      for p in l.parameters(): yield p

In [67]:
mdl = DummyModule(m, nh, 10)
mdl

AttributeError: ignored